# Define vibrational coordinates for OCS

Selecting appropriate vibrational __coordinates__ is crucial for accelerating variational calculations by improving the convergence of the __basis set__.

Examples of some commonly used vibrational coordinates for triatomic molecules.

| Coordinate system | Definition | 
|:-:|:-:|
|Valence-bond coordinates | <img src="assets/images/ocs_valence.png" alt="Alternative text" width="300" />|
|Jacobi coordinates |  <img src="assets/images/ocs_jacobi.png" alt="Alternative text" width="300" />|

Note: the orientation of the Cartesian axes $x$, $y$, and $z$ in the molecule does not really matter, as long as molecular rotation and rotational-vibrational coupling (Coriolis coupling) are disregarded.

## Start by loading necessary modules and functions

- `com`: shifts Cartesian coordianates of atoms to the centre of mass.
- `masses`: masses of atoms in the order 'C', 'O', 'S'.
- `jax`: library for computing derivatives of functions; will be necessary for computing the kinetic-energy operator.
- `jax.numpy` or `jnp`: numpy functions that can be differentiated using `jax`. Use it as standard numpy.

In [1]:
from kinetic import com
from molecule import masses
import jax
from jax import numpy as jnp
from jax import config

config.update("jax_enable_x64", True)

## Code Cartesian coordinates of atoms as function of **valence-bond** coordinates

 <img src="assets/images/ocs_valence.png" alt="Alternative text" width="200" />

In [2]:
@com
def _valence_to_cartesian(valence_coords):
    r_CO, r_CS, alpha = valence_coords
    return jnp.array(
        [
            [0.0, 0.0, 0.0],  # C
            [-r_CO * jnp.cos(alpha / 2), 0.0, -r_CO * jnp.sin(alpha / 2)],  # O
            [-r_CS * jnp.cos(alpha / 2), 0.0, r_CS * jnp.sin(alpha / 2)],  # S
        ],
        dtype=jnp.float64,
    )


def _cartesian_to_valence(xyz):
    n_CO = xyz[1] - xyz[0]
    n_CS = xyz[2] - xyz[0]
    r_CO = jnp.linalg.norm(n_CO)
    r_CS = jnp.linalg.norm(n_CS)
    alpha = jnp.arccos(jnp.dot(n_CO, n_CS) / (r_CO * r_CS))
    return jnp.array([r_CO, r_CS, alpha])


cartesian_to_valence = jax.vmap(_cartesian_to_valence, in_axes=(0,))
valence_to_cartesian = jax.vmap(_valence_to_cartesian, in_axes=(0,))

## Code Cartesian coordinates of atoms as function of **Jacobi** coordinates

<img src="assets/images/ocs_jacobi.png" alt="Alternative text" width="200" />

In [3]:
@com
def _jacobi_to_cartesian(jacobi_coords):
    r_S, r_CO, theta = jacobi_coords
    mass_C, mass_O, mass_S = masses
    z_C = 0
    z_O = r_CO
    z_com = (z_C * mass_C + z_O * mass_O) / (mass_C + mass_O)
    return jnp.array(
        [
            [0.0, 0.0, 0.0],  # C
            [0.0, 0.0, r_CO],  # O
            [r_S * jnp.sin(theta), 0.0, z_com + r_S * jnp.cos(theta)],  # S
        ],
        dtype=jnp.float64,
    )


def _cartesian_to_jacobi(xyz):
    n1 = xyz[1] - xyz[0]
    r_CO = jnp.linalg.norm(n1)
    com = jnp.dot(masses[:2], xyz[:2, :]) / jnp.sum(jnp.array(masses[:2]))
    n2 = xyz[2] - com
    r_S = jnp.linalg.norm(n2)
    theta = jnp.arccos(jnp.dot(n1, n2) / (r_CO * r_S))
    return jnp.array([r_S, r_CO, theta])


cartesian_to_jacobi = jax.vmap(_cartesian_to_jacobi, in_axes=(0,))
jacobi_to_cartesian = jax.vmap(_jacobi_to_cartesian, in_axes=(0,))